# Code: Global pairwise alignments

We will now to make use of our definitions of a Needleman-Wunsch alignment to see how the algorithm transforms to actual code. The sections below will walk you through how this is done. 

```{toggle}
To facilitate the the reasoning in the subsequent cells, we first we define a couple of service functions that we will need later, for formating and printing alignments. It is not important that you understand what these functions do, for now.
```

In [1]:
import numpy as np

# Print 2 sequences on top of each other
def print_alignment(seqA,seqB):
    print(seqA)
    print(seqB)

def traceback_alignment_latex(seqA, seqB, scores, traceback):
    """
    Generates LaTeX code for the matrix with traceback arrows.
    
    Parameters:
        seqA (str): First sequence.
        seqB (str): Second sequence.
        scores (numpy array): Score matrix.
        traceback (dict): Traceback matrix.
        
    Returns:
        latex_code (str): LaTeX code for the matrix with traceback arrows.
    """
    seqA, seqB = "-" + seqA, "-" + seqB
    nrows = len(seqA)
    ncols = len(seqB)
    latex_code = ""
        
    # Draw arrows for traceback paths
    def backtrack_latex(i, j):
        latex_code = ""
        for di, dj in traceback.get((i, j), []):
            ni, nj = i + di, j + dj
            if di == -1 and dj == -1:  # Diagonal arrow
                latex_code += f"\\draw [blue,very thick,->] ({j + 1.3},{nrows - i - 1.2}) -- ({j + 0.9},{nrows - i - 0.8});\n"
            elif di == -1:  # Upward arrow
                latex_code += f"\\draw [blue,very thick,->] ({j + 1.6},{nrows - i - 1.2}) -- ({j + 1.6},{nrows - i - 0.8});\n"
            elif dj == -1:  # Leftward arrow
                latex_code += f"\\draw [blue,very thick,->] ({j + 1.2},{nrows - i - 1.4}) -- ({j + 0.8},{nrows - i - 1.4});\n"                
            else:  # Stop
                return latex_code                
            latex_code += backtrack_latex(ni, nj)
        return latex_code
    
    # Start traceback from the bottom-right corner
    latex_code += backtrack_latex(nrows - 2, ncols - 2)
        
    return latex_code


# Format an alignment by inserting gaps in sequences
def format_alignment(seqA,seqB,S,trace):
    print("Best score: " + str(S[-1,-1]))
    outA,outB = "",""
    i,j = len(seqA),len(seqB)
    while i>0 or j>0:
        print(i,j,trace[(i,j)])
        di,dj = trace[(i,j)]
        i += int(di)
        j += int(dj)
        if di == 0:
            outA = "-" + outA
        else:
            outA = seqA[i] + outA
        if dj == 0:
            outB = "-" + outB
        else:
            outB = seqB[j] + outB
    return outA,outB

In [2]:
# redefine print statement so that it prints latex code
def print_dynamic(seqA, seqB, dpm, trace):
    """
    Generates LaTeX code for a matrix representation of the pairwise alignment.
    
    Parameters:
    - seqA: The first sequence (e.g., "GAC")
    - seqB: The second sequence (e.g., "ACG")
    - dpm: A 2D list or numpy array containing the alignment scores.
    - trace: A dictionary or 2D array where each element is a tuple (di, dj) indicating the direction of the path.
    
    Returns:
    - A string containing the LaTeX code.
    """
    seqA, seqB = "-" + seqA, "-" + seqB
    nrows = len(seqA)
    ncols = len(seqB)
    
    latex_code = "\\begin{tikzpicture}[set style={{help lines}+=[dashed]}, xscale=1.0, yscale=1]\n\n"
    latex_code += f"\\draw (1,0) grid +({ncols},{nrows});\n\n"
    
    # Labels for sequences
    for i, letter in enumerate(seqA):
        latex_code += f"\\node  at  (0.5, {nrows - i - 0.5}) {{{letter}}};\n"
    for j, letter in enumerate(seqB):
        latex_code += f"\\node  at  ({j + 1.5}, {nrows + 0.5}) {{{letter}}};\n"
    
    latex_code += "\n"
    
    # Fill matrix values and draw arrows if applicable
    for i in range(nrows):
        for j in range(ncols):
            value = dpm[i][j]
            latex_code += f"\\node  at  ({j + 1.5}, {nrows - i - 0.5}) {{{value}}};\n"
            if (i,j) in trace:
                for di, dj in trace[(i, j)]:
                    ii, jj = i+di, j+dj
                    #ii, jj = i, j
                    if di == -1 and dj == -1:
                        latex_code += f"\\draw   [red,very thick,dashed,->]   ({jj + 1.8},{nrows - ii - 0.8}) -- ({jj + 2.2},{nrows - ii - 1.2});\n"
                    elif di == -1 and dj == 0:
                        latex_code += f"\\draw   [red,very thick,dashed,->]   ({jj + 1.5},{nrows - ii - 0.8}) -- ({jj + 1.5},{nrows - ii - 1.2});\n"
                    elif di == 0 and dj == -1:
                        latex_code += f"\\draw   [red,very thick,dashed,->]   ({jj + 1.8},{nrows - ii - 0.5}) -- ({jj + 2.2},{nrows - ii - 0.5});\n"
                    elif di == 0 and dj == 0:
                        latex_code += f"\\draw   [red,very thick]   ({jj + 1.2},{nrows - ii - 0.2}) -- ({jj + 1.2},{nrows - ii - 0.3});\n"
    
    latex_code += traceback_alignment_latex(seqA, seqB, dpm, trace)
    latex_code += "\\end{tikzpicture}\n"
    
    print(latex_code)


## Scoring system for DNA sequences
We setup the scoring system we need for the alignment of DNA sequences. Here we use a score system where gaps score -2 and miss matches are scored -1 and matches get a score of 3. You should absolutly try to change the scoring function at some point to see how the values affects the alignments. 

In [3]:
def gap_penalty():
    return -2.0

def match_score(letterA,letterB):
    if letterA == '-' or letterB == '-':
        return gap_penalty()
    elif letterA == letterB:
        return 3.0
    else:
        return -1.0


## Global alignments by Needleman-Wunsch
Here follows an implementation of the [Needleman-Wunsch](http://www.sciencedirect.com/science/article/pii/0022283670900574) pairwise alignment method.  We want to align two sequences $a=a_1,\ldots,a_{M}$ and $b=b_1,\ldots,b_{N}$. 

As said in last chapter, the dynamic programming matrix $S$ is initiated as:
$S_{i0}=g \cdot i, \forall i,$
$S_{0j}=g \cdot j, \forall j$

This translates easy into two for loops filling in the upper row and the leftmost column.

Here we als initiate pointers in the form of a trace matrix. Each element in the trace matrix contains the difference in index between the current cell and the optimal step that lead to the current cell. 

In [4]:
# Initiating dynamic programming matrices, S and trace
def initiate_global_dp(m,n):
    S = np.zeros((m,n))       # An m*n matrix, initiated with 0's
    trace = {}
    S[0,0] = 0.
    trace[(0,0)] = [(0.,0.)]
    for i in range(1,m):
        S[i,0] = i * gap_penalty()
        trace[(i,0)] =[(-1,0)]
    for j in range(1,n):
        S[0,j] = j * gap_penalty()
        trace[(0,j)] =[(0,-1)]
    return S,trace

Subsequently, the rest of $S$ is filled as:
$S_{ij}=\max\left\{
\begin{array}{ll}
S_{i-1,j-1} & +d(a_i,b_j)\\
S_{i-1,j} & +d(a_i,-)\\
S_{i,j-1} & +d(-,b_j)
\end{array}
\right.$

This recursion is easily transformed into for-loops. We are free to select the order the matrix is filled in so we select to fill in row per row, i.e. rows become the inner loop, the columns the outer loop.

Ahgain we keep track of the move that lead to a certain position by filling in the `trace` matrix.


In [5]:
def global_align(seqA,seqB,print_dynamic_matrix = False):
    # Initiating variables
    m, n = len(seqA)+1, len(seqB)+1
    S,trace = initiate_global_dp(m,n)
    # Fill in the rest of the dynamic programming matrix
    for i in range(1,m):
        for j in range(1,n):
            # Note the subtraction of 1 for the sequence position
            # In python sequences are indexed from 0 to len-1
            match = S[i-1,j-1] + match_score(seqA[i-1],seqB[j-1]) 
            delete = S[i-1,j] + match_score(seqA[i-1],'-') 
            insert = S[i,j-1] + match_score('-',seqB[j-1]) 
            S[i,j] = max(match,delete,insert)
            trace[(i,j)] = [] 
            if match == S[i,j]:
                trace[(i,j)].append((-1,-1)) 
            if delete == S[i,j]:
                trace[(i,j)].append((-1,0)) 
            if insert == S[i,j]:
                trace[(i,j)].append((0,-1))
    if print_dynamic_matrix:
        print_dynamic(seqA,seqB,S,trace)
    return

Now everything is set. We can try the code for any of our favorite sequences. One can toggle the printout of the dynamic programming matrix by a boolean flag as a third argument.

In [6]:
global_align("GATTA","GCTAC",True)

\begin{tikzpicture}[set style={{help lines}+=[dashed]}, xscale=1.0, yscale=1]

\draw (1,0) grid +(6,6);

\node  at  (0.5, 5.5) {-};
\node  at  (0.5, 4.5) {G};
\node  at  (0.5, 3.5) {A};
\node  at  (0.5, 2.5) {T};
\node  at  (0.5, 1.5) {T};
\node  at  (0.5, 0.5) {A};
\node  at  (1.5, 6.5) {-};
\node  at  (2.5, 6.5) {G};
\node  at  (3.5, 6.5) {C};
\node  at  (4.5, 6.5) {T};
\node  at  (5.5, 6.5) {A};
\node  at  (6.5, 6.5) {C};

\node  at  (1.5, 5.5) {0.0};
\draw   [red,very thick]   (1.2,5.8) -- (1.2,5.7);
\node  at  (2.5, 5.5) {-2.0};
\draw   [red,very thick,dashed,->]   (1.8,5.5) -- (2.2,5.5);
\node  at  (3.5, 5.5) {-4.0};
\draw   [red,very thick,dashed,->]   (2.8,5.5) -- (3.2,5.5);
\node  at  (4.5, 5.5) {-6.0};
\draw   [red,very thick,dashed,->]   (3.8,5.5) -- (4.2,5.5);
\node  at  (5.5, 5.5) {-8.0};
\draw   [red,very thick,dashed,->]   (4.8,5.5) -- (5.2,5.5);
\node  at  (6.5, 5.5) {-10.0};
\draw   [red,very thick,dashed,->]   (5.8,5.5) -- (6.2,5.5);
\node  at  (1.5, 4.5) {-2.0};
\dra

I add a couple of extra alignments, check them manually as an excercise before the exam. Also try to run a couple of them yourself.

In [7]:
global_align("GCAGCTA","GCTA",True)


\begin{tikzpicture}[set style={{help lines}+=[dashed]}, xscale=1.0, yscale=1]

\draw (1,0) grid +(5,8);

\node  at  (0.5, 7.5) {-};
\node  at  (0.5, 6.5) {G};
\node  at  (0.5, 5.5) {C};
\node  at  (0.5, 4.5) {A};
\node  at  (0.5, 3.5) {G};
\node  at  (0.5, 2.5) {C};
\node  at  (0.5, 1.5) {T};
\node  at  (0.5, 0.5) {A};
\node  at  (1.5, 8.5) {-};
\node  at  (2.5, 8.5) {G};
\node  at  (3.5, 8.5) {C};
\node  at  (4.5, 8.5) {T};
\node  at  (5.5, 8.5) {A};

\node  at  (1.5, 7.5) {0.0};
\draw   [red,very thick]   (1.2,7.8) -- (1.2,7.7);
\node  at  (2.5, 7.5) {-2.0};
\draw   [red,very thick,dashed,->]   (1.8,7.5) -- (2.2,7.5);
\node  at  (3.5, 7.5) {-4.0};
\draw   [red,very thick,dashed,->]   (2.8,7.5) -- (3.2,7.5);
\node  at  (4.5, 7.5) {-6.0};
\draw   [red,very thick,dashed,->]   (3.8,7.5) -- (4.2,7.5);
\node  at  (5.5, 7.5) {-8.0};
\draw   [red,very thick,dashed,->]   (4.8,7.5) -- (5.2,7.5);
\node  at  (1.5, 6.5) {-2.0};
\draw   [red,very thick,dashed,->]   (1.5,7.2) -- (1.5,6.8);
\node  a

In [8]:
seqA,seqB = global_align("CTATCTCGCTATCCA","CTACGCTATTTCA",True)
print_alignment(seqA,seqB)

\begin{tikzpicture}[set style={{help lines}+=[dashed]}, xscale=1.0, yscale=1]

\draw (1,0) grid +(14,16);

\node  at  (0.5, 15.5) {-};
\node  at  (0.5, 14.5) {C};
\node  at  (0.5, 13.5) {T};
\node  at  (0.5, 12.5) {A};
\node  at  (0.5, 11.5) {T};
\node  at  (0.5, 10.5) {C};
\node  at  (0.5, 9.5) {T};
\node  at  (0.5, 8.5) {C};
\node  at  (0.5, 7.5) {G};
\node  at  (0.5, 6.5) {C};
\node  at  (0.5, 5.5) {T};
\node  at  (0.5, 4.5) {A};
\node  at  (0.5, 3.5) {T};
\node  at  (0.5, 2.5) {C};
\node  at  (0.5, 1.5) {C};
\node  at  (0.5, 0.5) {A};
\node  at  (1.5, 16.5) {-};
\node  at  (2.5, 16.5) {C};
\node  at  (3.5, 16.5) {T};
\node  at  (4.5, 16.5) {A};
\node  at  (5.5, 16.5) {C};
\node  at  (6.5, 16.5) {G};
\node  at  (7.5, 16.5) {C};
\node  at  (8.5, 16.5) {T};
\node  at  (9.5, 16.5) {A};
\node  at  (10.5, 16.5) {T};
\node  at  (11.5, 16.5) {T};
\node  at  (12.5, 16.5) {T};
\node  at  (13.5, 16.5) {C};
\node  at  (14.5, 16.5) {A};

\node  at  (1.5, 15.5) {0.0};
\draw   [red,very thick]   

TypeError: cannot unpack non-iterable NoneType object